In [10]:
%%html
<h1 style="color:orange;">INTRO TO BLOCK CHAIN ENGINEERING - PROJECT-11 Block chain of records - 12613141</h1>

In [1]:
!pip install faker

In [2]:
import json
from faker import Faker
import datetime

def generate_fake_record():
    """
    This method is helper function to generate fake profile using faker library such that object is json serialisable
    """
    fake = Faker()
    fake_datetime = fake.date_time()
    email = fake.email().split('@')[0] + fake.random_element(['@gmail.com', '@mst.edu'])
    record = {
        'username': fake.user_name(),
        'name': fake.name(),
        'sex': fake.random_element(['M', 'F']),
        'address': fake.address(),
        'mail': email,
        'birthdate': fake_datetime.strftime('%Y-%m-%d %H:%M:%S')
    }
    return record

# Generate record with datetime birthdate
record = generate_fake_record()
print(record)

{'username': 'jamiemora', 'name': 'Dr. Michael Rodriguez', 'sex': 'M', 'address': '50991 Brian Orchard\nSouth Colleen, IL 44423', 'mail': 'shannon23@mst.edu', 'birthdate': '1993-03-30 07:10:02'}


In [3]:
# source: https://github.com/karmus89/blockchain-a-z

import datetime
import hashlib
import json
import sys


class Blockchain:

    def __init__(self):

        self.chain = []
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        "Create a block."
        #Adding new field record to the existing block structure
        block = {
            'index': len(self.chain),
            'timestamp': str(datetime.datetime.now()),
            'proof': proof,
            'previous_hash': previous_hash,
            'record': generate_fake_record()
        }
        self.chain.append(block)

        return block

    def get_previous_block(self):

        return self.chain[-1]

    def hash_proof(self, previous_proof, next_proof):
        "Calculate the SHA256-hash"

        return (hashlib
                .sha256(str(next_proof**2 - previous_proof**2).encode())
                .hexdigest())

    def hash_block(self, block):
        "Calculate the SHA256-hash for a block."

        encoded_block = json.dumps(block, sort_keys=True).encode()

        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, previous_proof):
        "Calculate a new proof related to the previous block."

        next_proof = 1
        check_proof = False

        while check_proof is False:

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                check_proof = True

            else:
                next_proof += 1

        return next_proof

    def is_chain_valid(self, chain):
        "Validate that block and proof hashes are correct across the chain."

        for i in range(len(chain)):

            if i == 0:
                continue

            if chain[i]['previous_hash'] != self.hash_block(chain[i-1]):
                return False

            previous_proof = chain[i-1]['proof']
            next_proof = chain[i]['proof']

            if self.hash_proof(previous_proof, next_proof)[:4] != '0000':

                return False

        return True

In [4]:
import threading
import requests

from flask import Flask, request, jsonify
from werkzeug.serving import run_simple


class BlockchainApp:

    def __init__(self, host='localhost', port=5000, chain=Blockchain):

        self.host = host
        self.port = port
        self.chain = chain()

        self.host_url = f'http://{self.host}:{self.port}'

        self.app = Flask(__name__)
        self.add_api_endpoints()

        self.thread = threading.Thread(
            target=run_simple,
            kwargs={
                'hostname': self.host,
                'port': self.port,
                'application': self.app}
        )

    def __enter__(self):

        self.start()

        return self

    def __exit__(self, *args):

        self.stop()

    def add_api_endpoints(self):
        "Add API endpoints to the Flask WebApp."

        self.app.add_url_rule(
            rule='/blocks',
            view_func=self.api_blocks,
            methods=['GET', 'POST']
        )
        self.app.add_url_rule(
            rule='/blocks/validate',
            view_func=self.api_validate,
        )
        self.app.add_url_rule(
            rule='/shutdown',
            view_func=self.api_shutdown,
        )

    def api_blocks(self):
        "Either retrieve the node's current chain or post a new block to the chain."

        if request.method == 'POST':

            prev_block = self.chain.get_previous_block()
            prev_hash = self.chain.hash_block(prev_block)
            prev_proof = prev_block['proof']

            proof = self.chain.proof_of_work(prev_proof)

            block = self.chain.create_block(proof, prev_hash)

            response = {'message': 'Congratulations, you just mined a Block!',
                        'block': block}

            return (jsonify(response), 200)

        if request.method == 'GET':

            response = {'chain': self.chain.chain,
                        'length': len(self.chain.chain)}

            return (jsonify(response), 200)

    def api_validate(self):
        "Validate the chain"

        if self.chain.is_chain_valid(self.chain.chain):

            response = {'message': 'Chain is valid.',
                        'valid': True}
            return (jsonify(response), 200)

        else:

            response = {'message': 'Chain is not valid!',
                        'valid': False}
            return (jsonify(response), 500)

    def api_shutdown(self):
        "Shutdown the Flask WebApp"

        # depricated
        # request.environ.get('werkzeug.server.shutdown')()

        return jsonify({'message': 'Shutting down'}), 200

    def start(self):
        "Start the Flask-based Blockchain WebApp."

        self.thread.start()

    def stop(self):
        "Stop the Flask-based Blockchain WebApp."

        if self.thread.is_alive():

            return requests.get(f'{self.host_url}/shutdown')

In [5]:
"""
Helper functions to validate mail and date of births
"""

# Function to find names of people with gmail.com email from the blockchain
def find_gmail_users(blockchain):
    gmail_users = []
    for block in blockchain:
        if 'mail' in block['record'] and block['record']['mail'].endswith('@gmail.com'):
            gmail_users.append(block['record']['name'])
    return gmail_users

# Function to find usernames of people born on or after 2000-01-01 from the blockchain
def find_young_users(blockchain):
    young_users = []
    for block in blockchain:
        if 'record' in block and 'birthdate' in block['record'] and block['record']['birthdate'] >= '2000-01-01':
            young_users.append(block['record']['username'])
    return young_users

In [6]:
import time
# Test the modified blockchain node
with BlockchainApp() as blockchain_app:
    # Wait for server thread ready

    # a. Request to mine 30 blocks
    print("Mining 30 blocks...")
    for _ in range(30):
        response = requests.post(f'{blockchain_app.host_url}/blocks')
        print(f"Proof: {response.json()['block']['proof']}")
    # Print the resulting blockchain after mining

    response = requests.get(f'{blockchain_app.host_url}/blocks')
    print("Chain={}".format(json.dumps(response.json(), indent=2)))

    for i in enumerate(blockchain_app.chain.chain):
      print(i)
    # b. Find names of all people who have gmail.com email from the blockchain
    print("\nNames of people with gmail.com email:")
    for name in find_gmail_users(blockchain_app.chain.chain):
        print(name)
    # c. Find usernames of people born on or after 2000-01-01 from the blockchain
    print("\nUsernames of people born on or after 2000-01-01:")
    for username in find_young_users(blockchain_app.chain.chain):
        print(username)
    # d. Request to validate the blockchain
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print("\nValidation result after mining 30 blocks:", response.json())
    # e. Create a new profile record and overwrite the 3rd block's record field with it
    new_record = generate_fake_record()
    blockchain_app.chain.chain[2]['record'] = [new_record]
    # f. Request to validate the altered blockchain
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print("\nValidation result after altering the 3rd block's record:", response.json())

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:05] "POST /blocks HTTP/1.1" 200 -


Mining 30 blocks...
Proof: 533


INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:06] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:06] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:06] "POST /blocks HTTP/1.1" 200 -


Proof: 45293
Proof: 21391
Proof: 8018


INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:06] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:06] "POST /blocks HTTP/1.1" 200 -


...

Proof: 41769
Proof: 37872


INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:11] "POST /blocks HTTP/1.1" 200 -


Proof: 56560


INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:12] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:12] "GET /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:12] "GET /blocks/validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:12] "GET /blocks/validate HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 00:07:12] "GET /shutdown HTTP/1.1" 200 -


Proof: 93290
Chain={
  "chain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "record": {
        "address": "5751 Mejia Walks Apt. 019\nSouth Staciefurt, MO 77029",
        "birthdate": "2009-06-03 06:45:46",
        "mail": "nwoodard@gmail.com",
        "name": "Michael Adkins",
        "sex": "M",
        "username": "williamnewman"
      },
      "timestamp": "2024-04-30 00:07:05.799924"
    },
    {
      "index": 1,
      "previous_hash": "b5232e8d64d14eac4f8fe91031858be45b2c12a8b8f8100be7c32c3e324ea80b",
      "proof": 533,
      "record": {
        "address": "3951 Kelly Pike Suite 467\nMartinberg, ID 55766",
        "birthdate": "1981-10-02 12:08:28",
        "mail": "cpugh@mst.edu",
        "name": "Steven Bradley",
        "sex": "F",
        "username": "mortonelizabeth"
      },
      "timestamp": "2024-04-30 00:07:05.871674"
    },
    {
      "index": 2,
      "previous_hash": "dc90e02e6472db97182c1d18787439aa28012f5a68db65effa87930a9b55d1

In [7]:
!apt install net-tools
!netstat -tulnp | grep :5000

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 204 kB of archives.
After this operation, 819 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 net-tools amd64 1.60+git20181103.0eebece-1ubuntu5 [204 kB]
Fetched 204 kB in 1s (328 kB/s)
Selecting previously unselected package net-tools.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../net-tools_1.60+git20181103.0eebece-1ubuntu5_amd64.deb ...
Unpacking net-tools (1.60+git20181103.0eebece-1ubuntu5) ...
Setting up net-tools (1.60+git20181103.0eebece-1ubuntu5) ...
Processing triggers for man-db (2.10.2-1) ...
tcp        0      0 127.0.0.1:5000          0.0.0.0:*               LISTEN      28792/python3       


In [8]:
!kill 703

/bin/bash: line 1: kill: (703) - No such process


In [11]:
%%html
<h1 style="color:orange;">END OF PROJECT</h1>